# Day 1: Image Generation Pipeline

**What this notebook does:**
1. Installs dependencies on Colab's free GPU
2. Parses a crash report into structured data (using Groq API)
3. Generates a photorealistic dashcam image from the parsed scenario (using **SDXL**)

**Requirements:**
- Google Colab with GPU runtime (`Runtime → Change runtime type → T4 GPU`)
- Groq API key (free at console.groq.com)
- Project files uploaded to Google Drive

**Model:** Stable Diffusion XL (SDXL) — 1024x1024 resolution, much better prompt adherence than SD 1.5

## Step 1: Setup Environment

In [ ]:
# Install dependencies (Colab already has torch + CUDA)
!pip install -q diffusers>=0.25.0 transformers accelerate safetensors
!pip install -q openai pydantic python-dotenv

In [ ]:
# Mount Google Drive to access project files
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Add project to Python path
import sys
sys.path.append('/content/drive/MyDrive/tesla_crash_synth')

# Verify imports work
from config import CrashScenario, Weather, IncidentType
from utils.parser import LLMParser
from utils.image_generator import BasicImageGenerator
print('All imports successful!')

In [ ]:
# Check GPU is available
import torch
print(f'PyTorch: {torch.__version__}')
print(f'CUDA available: {torch.cuda.is_available()}')
if torch.cuda.is_available():
    print(f'GPU: {torch.cuda.get_device_name(0)}')
    print(f'VRAM: {torch.cuda.get_device_properties(0).total_mem / 1e9:.1f} GB')

## Step 2: Set API Key

Enter your Groq API key below. Get one free at [console.groq.com](https://console.groq.com)

In [ ]:
import os

# Option A: Paste your key directly (quick testing)
os.environ['GROQ_API_KEY'] = 'YOUR_KEY_HERE'  # <-- Replace this

# Option B: Or load from .env file on Drive
# from dotenv import load_dotenv
# load_dotenv('/content/drive/MyDrive/tesla_crash_synth/.env')

## Step 3: Parse a Crash Report

In [ ]:
# Initialize parser
parser = LLMParser()

# Parse a crash report
report = "Vehicle traveling 45mph on wet highway, hydroplaned into guardrail"
scenario = parser.parse(report)

# Show parsed result
print(scenario.model_dump_json(indent=2))
print(f'\n--- Image Prompt Sent to SDXL ---')
print(scenario.image_prompt)

## Step 4: Generate Dashcam Image

This downloads the SDXL model (~6.5GB) on first run. After that it's cached.

SDXL generates **1024x1024** images (4x more pixels than SD 1.5's 512x512) with much better prompt adherence.

In [ ]:
# Initialize image generator (downloads model on first run)
generator = BasicImageGenerator()  # auto-detects CUDA on Colab

In [ ]:
# Generate image from parsed scenario
image = generator.generate_from_scenario(scenario)

# Display in notebook
display(image)

In [ ]:
# Save to Drive
output_dir = '/content/drive/MyDrive/tesla_crash_synth/outputs'
os.makedirs(output_dir, exist_ok=True)
image.save(f'{output_dir}/day1_hydroplane.png')
print(f'Saved to {output_dir}/day1_hydroplane.png')

## Step 5: Batch Generate Multiple Scenarios

Let's test the full pipeline with different crash reports.

In [ ]:
reports = [
    "Vehicle traveling 45mph on wet highway, hydroplaned into guardrail",
    "Car rear-ended truck at stoplight in clear weather",
    "Pedestrian crossed outside crosswalk at night, struck by vehicle going 25mph",
    "Side impact at intersection, vehicle ran red light at 35mph",
]

for i, report in enumerate(reports):
    print(f'\n{"="*60}')
    print(f'Report {i+1}: {report}')
    
    # Parse
    scenario = parser.parse(report)
    print(f'Parsed: {scenario.incident_type}, {scenario.weather}, {scenario.ego_speed_mps:.1f} m/s')
    print(f'SD Prompt: {scenario.image_prompt[:100]}...')
    
    # Generate
    image = generator.generate_from_scenario(scenario)
    
    # Save
    path = f'{output_dir}/day1_test_{i}.png'
    image.save(path)
    print(f'Saved: {path}')
    
    # Display
    display(image)

print(f'\nDone! {len(reports)} images saved to {output_dir}')

## Results Summary

If everything worked, you should have:
- 4 dashcam images in `outputs/` on your Drive
- Each matching its crash report description (weather, road type, incident)

**Next step:** Day 2 — ControlNet for depth-controlled scene composition